# Subspace Digital Twin, Initial Conditions Run

*Shawn Anderson, January 2024*

In this notebook, we examine medianl behavior over the first 90 days.

## Part 1. Dependences & Set-up

Autoreload modules while developing.

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import numpy as np
import pandas as pd
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)

import hvplot.pandas
hvplot.extension('bokeh')

from bokeh.models import HoverTool
import holoviews as hv

from bokeh.palettes import Turbo256, Category20

## Part 2. Load Simulation Data

Load the simulation results data.

In [3]:
sim_df = pd.read_pickle(
    "../data/simulations/reference_subsidy_sweep-2024-01-10_12-04-38.pkl.gz"
).drop(['timestep', 'simulation', 'subset', 'timestep_in_days', 'block_time_in_seconds', 'delta_days', 'delta_blocks'], axis=1)

In [4]:
sim_df.head(5)

,days_passed,blocks_passed,circulating_supply,user_supply,earned_supply,issued_supply,earned_minus_burned_supply,total_supply,sum_of_stocks,block_utilization,dsf_relative_disbursal_per_day,reward_issuance_balance,other_issuance_balance,operators_balance,nominators_balance,holders_balance,farmers_balance,staking_pool_balance,fund_balance,burnt_balance,nominator_pool_shares,operator_pool_shares,block_reward,history_size,space_pledged,allocated_tokens,buffer_size,reference_subsidy,average_base_fee,average_priority_fee,average_compute_weight_per_tx,average_transaction_size,transaction_count,average_compute_weight_per_bundle,average_bundle_size,bundle_count,compute_fee_volume,storage_fee_volume,rewards_to_nominators,run,average_compute_weight_per_budle,label,environmental_label,max_credit_supply
0,0,0.0,0.000000,0.000000,0.0,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,0.0,1.320000e+09,1.680000e+09,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,NaN,0,0,0.0,0.0,0.00000,0.0,0.0,0.0,256,0.000000e+00,0.0,0.0,0.0,0.000000,0.000000,0.0,1,NaN,constant-single-component,standard,3000000000
14,1,14400.0,13.698630,13.698630,0.0,1.680000e+09,0.0,1.680000e+09,3.000000e+09,0.5,0.0,1.320000e+09,1.680000e+09,0.000000,0.000000,0.684932,13.013699,0.0,0.000000,0.0,0.0,0.0,13.698630,2038498852864,1572944000000,0.0,60045568.0,13.69863,1.0,3.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.000000,0.000000,0.0,1,1.000000e+10,constant-single-component,standard,3000000000
28,2,28800.0,27.363014,27.397260,0.0,1.680000e+09,0.0,1.680000e+09,3.000000e+09,0.5,0.0,1.320000e+09,1.680000e+09,0.000000,0.000000,1.368151,25.994863,0.0,0.034247,0.0,0.0,0.0,13.698630,4076997705728,3145888000000,0.0,120091136.0,13.69863,1.0,3.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.342466,0.342466,0.0,1,1.000000e+10,constant-single-component,standard,3000000000
42,3,43200.0,41.201036,41.303690,0.0,1.680000e+09,0.0,1.680000e+09,3.000000e+09,0.5,0.0,1.320000e+09,1.680000e+09,0.000000,0.000000,2.060052,39.140985,0.0,0.102654,0.0,0.0,0.0,13.906430,6115764994048,4718832000000,0.0,45918976.0,13.69863,1.0,3.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.684075,0.684075,0.0,1,1.000000e+10,constant-single-component,standard,3000000000
56,4,57600.0,55.211744,55.417401,0.0,1.680000e+09,0.0,1.680000e+09,3.000000e+09,0.5,0.0,1.320000e+09,1.680000e+09,0.000745,0.000745,2.829882,52.380372,0.0,0.205657,0.0,0.0,0.0,14.113711,8154263846912,6291776000000,0.0,105964544.0,13.69863,1.0,3.0,60000000.0,256,3.981312e+09,0.0,0.0,86400.0,0.955515,1.030026,0.0,1,1.000000e+10,constant-single-component,standard,3000000000


Simulation Runs.

In [5]:
sim_df.groupby(['run', 'label', 'environmental_label']).size().reset_index(name='Days').head()

,run,label,environmental_label,Days
0,1,constant-single-component,standard,185
1,2,hybrid-single-component,standard,185
2,3,hybrid-two-components,standard,185


### Coloring Metrics
Use a constant mapping from columns to colors

In [6]:
color_palette = Category20
# columns_to_color = sorted(list(set(sim_df.columns) - {'environmental_label', 'label', 'run', 'blocks_passed', 'days_passed'}))
columns_to_color = sim_df.columns
if color_palette == Turbo256:
    column_colors = dict(zip(columns_to_color, [color_palette[int(i)] for i in np.linspace(0,len(color_palette)-1, len(columns_to_color))]))

if color_palette == Category20:
    column_colors = {col: Category20[20][i%20] for i, col in enumerate(columns_to_color)}


sim_df.count().to_frame().T.hvplot.bar(y=columns_to_color, color=[column_colors[c] for c in columns_to_color], rot=90, width=1400, height=500, title='Column Color Map', fontscale=1.4, yaxis=None)

:Bars   [index,Variable]   (value)

In [7]:
def snake_to_title(s):
    """Utility function used for printing chart titles and labels as Title Case.
    Example:
    snake_to_caps('snake_case')
    >>> 'Snake Case'
    """
    
    return ' '.join(word.capitalize() for word in s.split('_'))

def fan_chart_quantile_median(df, column='circulating_supply', median_only=False):
    """Combine an area chart of min-max and a line chart of median for a series."""

    # min, max, median
    fan_df = df.groupby('days_passed')[column].agg(['min', 'max', 'median'])

    opts = dict(width=1200, height=500, title=f'{snake_to_title(column)} Fan Chart', ylabel=f'{column}_min_max_median')

    # Median curve
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Median', '@median{0,0.00}')])
    median_chart = fan_df.hvplot(x='days_passed', y='median', alpha=1, line_width=4, label=f'{snake_to_title(column)} Median', tools=[hover], color=column_colors[column]).opts(**opts)
    if median_only:
        return median_chart

    # min-max band
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Days Passed', '$x{0,0}')])
    bands_chart = fan_df.hvplot.area(x='days_passed', y='min', y2='max', legend='top_left', alpha=0.4, tools=[hover], ylim=(0,None), color=column_colors[column]).opts(**opts)

    # Composition
    chart = bands_chart * median_chart
    return chart


def fan_chart_quantile(df, column='circulating_supply', median_only=False):
    """Combine an area chart of min-max and a line chart of quantile for a series."""

    # 25%, 50%, 75%
    fan_df = df.groupby('days_passed')[column].quantile([0.25, 0.5, 0.75]).unstack().rename(columns={0.50:'median', 0.25:'0.25',0.75:'0.75'})

    # return fan_df

    opts = dict(width=1200, height=500, title=f'{snake_to_title(column)} Quantile Fan Chart', ylabel=f'{column}_quantile')

    # Quantile curve
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Median', '@median{0,0.00}')])
    quatile_chart = fan_df.hvplot(x='days_passed', y='median', alpha=1, line_width=4, label=f'{snake_to_title(column)} Quantile', tools=[hover], color=column_colors[column]).opts(**opts)
    if median_only:
        return quatile_chart

    # min-max band
    hover = HoverTool(tooltips=[(f'{snake_to_title(column)} Days Passed', '$x{0,0}')])
    bands_chart = fan_df.hvplot.area(x='days_passed', y='0.25', y2='0.75', legend='top_left', alpha=0.4, tools=[hover], ylim=(0,None), color=column_colors[column]).opts(**opts)

    # Composition
    chart = bands_chart * quatile_chart
    return chart


In [9]:
system_balances = ['other_issuance_balance', 'reward_issuance_balance']
agent_balances = [
    'farmers_balance',
    'operators_balance',
    'user_supply',
    'nominators_balance',
]
agent_pool_balances = ['staking_pool_balance']
protocol_treasury_balances = ['fund_balance']
other_balances = list(set([c for c in sim_df.columns if 'balance' in c]) - set(system_balances + agent_balances + agent_pool_balances + protocol_treasury_balances) )
supply_columns = list({c for c in sim_df.columns if 'supply' in c} - {'max_credit_supply', 'issued_supply', 'total_supply'})
balance_columns = list(set([c for c in sim_df.columns if 'balance' in c]) - set(system_balances))

In [12]:
box_df = sim_df.set_index(['days_passed', 'label'])[balance_columns]
box_df.describe()

,fund_balance,staking_pool_balance,farmers_balance,burnt_balance,nominators_balance,operators_balance,holders_balance
count,555.000000,555.000000,555.000000,555.000000,555.000000,555.000000,555.000000
mean,821.217333,37.477973,2530.585467,0.140061,54.217781,18.262442,258.681382
std,817.950764,43.648477,1652.077357,0.196467,49.514348,13.872263,171.751254
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,129.658052,2.587962,1181.400399,0.002493,11.047337,6.271811,118.239567
50%,557.549602,19.559475,2160.805712,0.043680,41.209783,15.607416,221.116090
75%,1226.934664,57.718636,4017.654500,0.205256,86.268773,30.215930,412.985247
max,2944.939900,164.007019,5644.475529,0.805923,169.498502,45.187272,580.032701


In [13]:
box_df.reset_index().drop('days_passed',axis=1).melt(id_vars=['label']).hvplot.violin(y='value', by='variable', c='variable', groupby='label', legend='top_right', width=1200, height=500, title=f'SSC Balances Violin Chart Across Weeks by Run', cmap=column_colors, ylim=(0,box_df.max().max()*0.75))

:DynamicMap   [label]
   :Violin   [variable]   (value)

In [45]:
hv.HoloMap(sim_df.groupby('label').apply(lambda df: hv.Overlay([fan_chart_quantile(df, c) for c in balance_columns])).to_dict())

:HoloMap   [Default]
   :Overlay
      .Area.I                              :Area   [days_passed]   (0.25,0.75)
      .Curve.Fund_Balance_Quantile         :Curve   [days_passed]   (median)
      .Area.II                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Staking_Pool_Balance_Quantile :Curve   [days_passed]   (median)
      .Area.III                            :Area   [days_passed]   (0.25,0.75)
      .Curve.Farmers_Balance_Quantile      :Curve   [days_passed]   (median)
      .Area.IV                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Burnt_Balance_Quantile        :Curve   [days_passed]   (median)
      .Area.V                              :Area   [days_passed]   (0.25,0.75)
      .Curve.Nominators_Balance_Quantile   :Curve   [days_passed]   (median)
      .Area.VI                             :Area   [days_passed]   (0.25,0.75)
      .Curve.Operators_Balance_Quantile    :Curve   [days_passed]   (median)
      .Area.VII                            :Area   [days_passed]   (0.25,0.75)
      .Curve.Holders_Balance_Quantile      :Curve   [days_passed]   (median)

In [42]:
hv.Overlay(sim_df.groupby('label').apply(lambda df: hv.Overlay([fan_chart_quantile(df, c) for c in balance_columns])).to_dict().values())

:Overlay
   .Area.I                              :Area   [days_passed]   (0.25,0.75)
   .Curve.Fund_Balance_Quantile         :Curve   [days_passed]   (median)
   .Area.II                             :Area   [days_passed]   (0.25,0.75)
   .Curve.Staking_Pool_Balance_Quantile :Curve   [days_passed]   (median)
   .Area.III                            :Area   [days_passed]   (0.25,0.75)
   .Curve.Farmers_Balance_Quantile      :Curve   [days_passed]   (median)
   .Area.IV                             :Area   [days_passed]   (0.25,0.75)
   .Curve.Burnt_Balance_Quantile        :Curve   [days_passed]   (median)
   .Area.V                              :Area   [days_passed]   (0.25,0.75)
   .Curve.Nominators_Balance_Quantile   :Curve   [days_passed]   (median)
   .Area.VI                             :Area   [days_passed]   (0.25,0.75)
   .Curve.Operators_Balance_Quantile    :Curve   [days_passed]   (median)
   .Area.VII                            :Area   [days_passed]   (0.25,0.75)
   .Curve.Holders_Balance_Quantile      :Curve   [days_passed]   (median)
   .Area.VIII                           :Area   [days_passed]   (0.25,0.75)
   .Curve.I                             :Curve   [days_passed]   (median)
   .Area.IX                             :Area   [days_passed]   (0.25,0.75)
   .Curve.II                            :Curve   [days_passed]   (median)
   .Area.X                              :Area   [days_passed]   (0.25,0.75)
   .Curve.III                           :Curve   [days_passed]   (median)
   .Area.XI                             :Area   [days_passed]   (0.25,0.75)
   .Curve.IV                            :Curve   [days_passed]   (median)
   .Area.XII                            :Area   [days_passed]   (0.25,0.75)
   .Curve.V                             :Curve   [days_passed]   (median)
   .Area.XIII                           :Area   [days_passed]   (0.25,0.75)
   .Curve.VI                            :Curve   [days_passed]   (median)
   .Area.XIV                            :Area   [days_passed]   (0.25,0.75)
   .Curve.VII                           :Curve   [days_passed]   (median)
   .Area.XV                             :Area   [days_passed]   (0.25,0.75)
   .Curve.VIII                          :Curve   [days_passed]   (median)
   .Area.XVI                            :Area   [days_passed]   (0.25,0.75)
   .Curve.IX                            :Curve   [days_passed]   (median)
   .Area.XVII                           :Area   [days_passed]   (0.25,0.75)
   .Curve.X                             :Curve   [days_passed]   (median)
   .Area.XVIII                          :Area   [days_passed]   (0.25,0.75)
   .Curve.XI                            :Curve   [days_passed]   (median)
   .Area.XIX                            :Area   [days_passed]   (0.25,0.75)
   .Curve.XII                           :Curve   [days_passed]   (median)
   .Area.XX                             :Area   [days_passed]   (0.25,0.75)
   .Curve.XIII                          :Curve   [days_passed]   (median)
   .Area.XXI                            :Area   [days_passed]   (0.25,0.75)
   .Curve.XIV                           :Curve   [days_passed]   (median)

In [40]:
hv.Overlay([fan_chart_quantile(sim_df, c, median_only=False) for c in balance_columns]).opts(title='SSC Daily Balances Fan Chart Comparison', ylabel='SSC', legend_opts={'location':'top_left'})

:Overlay
   .Area.I                              :Area   [days_passed]   (0.25,0.75)
   .Curve.Fund_Balance_Quantile         :Curve   [days_passed]   (median)
   .Area.II                             :Area   [days_passed]   (0.25,0.75)
   .Curve.Staking_Pool_Balance_Quantile :Curve   [days_passed]   (median)
   .Area.III                            :Area   [days_passed]   (0.25,0.75)
   .Curve.Farmers_Balance_Quantile      :Curve   [days_passed]   (median)
   .Area.IV                             :Area   [days_passed]   (0.25,0.75)
   .Curve.Burnt_Balance_Quantile        :Curve   [days_passed]   (median)
   .Area.V                              :Area   [days_passed]   (0.25,0.75)
   .Curve.Nominators_Balance_Quantile   :Curve   [days_passed]   (median)
   .Area.VI                             :Area   [days_passed]   (0.25,0.75)
   .Curve.Operators_Balance_Quantile    :Curve   [days_passed]   (median)
   .Area.VII                            :Area   [days_passed]   (0.25,0.75)
   .Curve.Holders_Balance_Quantile      :Curve   [days_passed]   (median)

Next, slice individual columns.